# Get Workspace

In [ ]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset
from azureml.widgets import RunDetails

# Get Azure ML Workspace
ws = Workspace.from_config()


# Create/Get Compute Target

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_name = "aml-compute"
vm_size = "STANDARD_DS12_V2"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,  # STANDARD_NC6 is GPU-enabled
                                                                min_nodes=0,
                                                                max_nodes=1)
    # create the compute target
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

# Setup Train Environment 

In [ ]:
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.runconfig import DockerConfiguration
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 
from azureml.core.runconfig import DockerConfiguration

# create a new runconfig object
aml_run_config = RunConfiguration()

# enable Docker 
training_docker_config = DockerConfiguration(use_docker=True)
aml_run_config.docker = training_docker_config

# set Docker base image to the default CPU-based image
aml_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
aml_run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn'],pip_packages=['azureml-sdk'])


# Setup Train Pipeline

In [ ]:
from azureml.pipeline.steps import PythonScriptStep
train_source_dir = "./train"
train_entry_point = "train.py" 
   
train_step = PythonScriptStep(
    name="Train Model",
    script_name=train_entry_point,
    source_directory=train_source_dir,
    compute_target=compute_target,
    runconfig=aml_run_config,
    allow_reuse=False
)

# Setup Deployment Environment

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

conda_run_config = RunConfiguration(framework="python")
conda_run_config.docker = DockerConfiguration(use_docker=True)
conda_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

cd = CondaDependencies.create(pip_packages=['azureml-sdk','azureml-widgets','azureml-contrib-notebook','ipython','notebook','ipython_genutils','Jinja2<=3.0.0'])
conda_run_config.environment.python.conda_dependencies = cd

# Setup Deployment Pipeline

In [ ]:
from azureml.contrib.notebook import NotebookRunConfig, AzureMLNotebookHandler
from azureml.contrib.notebook import NotebookRunnerStep

iotdeployemnt_source_dir = "./deployment"
iotdeployemnt_entry_point = "deployment.ipynb"

handler = AzureMLNotebookHandler(timeout=600, progress_bar=False, log_output=True)

cfg = NotebookRunConfig(source_directory=iotdeployemnt_source_dir, notebook=iotdeployemnt_entry_point,
                        handler = handler,
                        parameters={},
                        run_config=conda_run_config)

notebook_runner_step = NotebookRunnerStep(name="iothub_deployment_step",
                                          notebook_run_config=cfg,
                                          allow_reuse=True,
                                          compute_target=compute_target)#,
                                          #output_notebook_pipeline_data_name="notebook_result")

# Create Pipeline

In [ ]:
from azureml.pipeline.core import Pipeline, StepSequence

step_sequence = StepSequence(steps=[train_step,notebook_runner_step])
pipeline1 = Pipeline(workspace=ws, steps=step_sequence)
pipeline1.validate()

# Run Pipeline

In [ ]:
pipeline_run1 = Experiment(ws, 'anomaly-detection').submit(pipeline1, regenerate_outputs=False)
print("Pipeline is submitted for execution")